# ChiselDB：调试友好的结构化数据库

ChiselDB 是一个用于辅助功能/性能调试的结构化数据库。它使用 DPI-C 接口将 Chisel Bundle 的数据记录到 sqlite 数据库中。

相比 verilog wire，Chisel Bundle 具有更高层的语义信息（可以类比成数个 verilog wire 组成的组），后续使用 SQL 查询语句访问这些结构化的数据进行分析。
```

我们提供了一个预先编译的 `emu-cdb-err` 仿真程序，其存在一个人为注入的 bug，将从 L2 Cache 到 L3 Cache 释放数据都置为常量值。

通过 `--dump-db` 参数启用 ChiselDB，同时启用 difftest，运行后 difftest 报错，并在 `./build` 路径下生成了 `.db` 文件。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd ${NOOP_HOME}

rm -f ./build/*.db

${READY2RUN_HOME}/emu-cdb-err \
    -i $NOOP_HOME/ready-to-run/linux.bin \
    --diff $NOOP_HOME/ready-to-run/riscv64-nemu-interpreter-so \
    --dump-db \
    2>linux.err || true

echo -n "Dump DB: "
realpath ./build/*.db

随后我们可以使用 sqlite 读取 db 文件并进行分析，下面查询所有地址为 `0x800419c0` 的 tilelink transaction，并使用 `./scripts/cache/convert_tllog.sh` 脚本来格式化输出。

In [ ]:
%%bash
cd ../ && source env.sh >/dev/null
cd ${NOOP_HOME}

DB=$(ls -t ./build/*db | head -n 1)
sqlite3 $DB "select * from TLLog where ADDRESS=0x800419c0" | sh ./scripts/cache/convert_tllog.sh